TRANSFER LEARNING -> with this approach we can use existing model (weights and bias) to train new model

BASE_MODEL -> Model to identify digits

NEW_MODEL -> Model to classify digits as even or odd
                       

   

Building the base model (model to identify digits)

In [24]:
## import req libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [25]:
## loading dataset
mnist= tf.keras.datasets.mnist
(X_train_full, Y_train_full), (X_test, Y_test)= mnist.load_data()

In [26]:
X_train_full.shape, Y_train_full.shape

((60000, 28, 28), (60000,))

In [27]:
X_valid, X_train=X_train_full[0:6000]/255. ,X_train_full[6000:]/255.
Y_valid, Y_train=Y_train_full[0:6000],Y_train_full[6000:]
X_test= X_test/255.

validation_set= (X_valid, Y_valid)

In [28]:
## building base model
LAYERS= [tf.keras.layers.Flatten(input_shape=[28,28], name="Input_Layer"),
         tf.keras.layers.Dense(350, activation="relu", kernel_initializer="he_uniform", name="Hidden_Layer_1"),
         tf.keras.layers.Dense(75, activation="relu", kernel_initializer="he_uniform", name="Hidden_Layer_2"),
         tf.keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_uniform", name="Output_Layer") ]

base_model= tf.keras.models.Sequential(LAYERS)

base_model.compile(loss="sparse_categorical_crossentropy", optimizer="SGD", metrics="accuracy")

base_model.fit(X_train,Y_train, validation_data=validation_set, epochs=30)


Epoch 1/30
1688/1688 [==============================] - 5s 3ms/step - loss: 0.5803 - accuracy: 0.8472 - val_loss: 0.3011 - val_accuracy: 0.9178
Epoch 2/30
1688/1688 [==============================] - 5s 3ms/step - loss: 0.2806 - accuracy: 0.9201 - val_loss: 0.2320 - val_accuracy: 0.9350
Epoch 3/30
1688/1688 [==============================] - 4s 3ms/step - loss: 0.2286 - accuracy: 0.9344 - val_loss: 0.2007 - val_accuracy: 0.9442
Epoch 4/30
1688/1688 [==============================] - 4s 3ms/step - loss: 0.1950 - accuracy: 0.9444 - val_loss: 0.1761 - val_accuracy: 0.9502
Epoch 5/30
1688/1688 [==============================] - 4s 3ms/step - loss: 0.1703 - accuracy: 0.9502 - val_loss: 0.1557 - val_accuracy: 0.9540
Epoch 6/30
1688/1688 [==============================] - 5s 3ms/step - loss: 0.1502 - accuracy: 0.9571 - val_loss: 0.1427 - val_accuracy: 0.9588
Epoch 7/30
1688/1688 [==============================] - 5s 3ms/step - loss: 0.1345 - accuracy: 0.9615 - val_loss: 0.1328 - val_accuracy:

In [29]:
## evaluating and saving the model
base_model.evaluate(X_test, Y_test)
base_model.save("base_model.h5")

313/313 [==============================] - 1s 2ms/step - loss: 0.0717 - accuracy: 0.9783


Building new model (using base model)

In [30]:
## loading base model
model= tf.keras.models.load_model("base_model.h5")

In [31]:
## changing the layers of base model to non-trainable i.e freezing the weights
for layer in model.layers[:-1]:
  print(f"Before freezing {layer.name} : {layer.trainable}")
  layer.trainable=False
  print(f"After freezing {layer.name}: {layer.trainable}")

Before freezing Input_Layer : True
After freezing Input_Layer: False
Before freezing Hidden_Layer_1 : True
After freezing Hidden_Layer_1: False
Before freezing Hidden_Layer_2 : True
After freezing Hidden_Layer_2: False


In [32]:
def even_odd(input_array):
  return np.where(input_array%2==0, 1,0) ## 1 for even 0 for odd

Y_train=even_odd(Y_train)
Y_valid= even_odd(Y_valid)
Y_test=even_odd(Y_test)

In [33]:
## Building new model:
new_model= tf.keras.models.Sequential(model.layers[:-1])  ## using base model layers

new_model.add(tf.keras.layers.Dense(2,activation="softmax", name="Output_Layer"))

new_model.layers

new_model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_Layer (Flatten)       (None, 784)               0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 350)               274750    
                                                                 
 Hidden_Layer_2 (Dense)      (None, 75)                26325     
                                                                 
 Output_Layer (Dense)        (None, 2)                 152       
                                                                 
Total params: 301,227
Trainable params: 152
Non-trainable params: 301,075
_________________________________________________________________


In [35]:

new_model.compile(loss="sparse_categorical_crossentropy", optimizer="SGD", metrics="accuracy")

new_model.fit(X_train,Y_train, validation_data=validation_set, epochs=20)

Epoch 1/20
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0978 - accuracy: 0.9643 - val_loss: nan - val_accuracy: 0.0050
Epoch 2/20
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0976 - accuracy: 0.9644 - val_loss: nan - val_accuracy: 0.0045
Epoch 3/20
1688/1688 [==============================] - 4s 3ms/step - loss: 0.0975 - accuracy: 0.9648 - val_loss: nan - val_accuracy: 0.0047
Epoch 4/20
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0973 - accuracy: 0.9644 - val_loss: nan - val_accuracy: 0.0052
Epoch 5/20
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0971 - accuracy: 0.9645 - val_loss: nan - val_accuracy: 0.0060
Epoch 6/20
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0968 - accuracy: 0.9648 - val_loss: nan - val_accuracy: 0.0048
Epoch 7/20
1688/1688 [==============================] - 4s 3ms/step - loss: 0.0969 - accuracy: 0.9649 - val_loss: nan - val_accuracy: 0.0050
Epoch 8/20
16

In [36]:


## evaluating new model 
new_model.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.1096 - accuracy: 0.9596


[0.10955099761486053, 0.9595999717712402]